# tsam - Representation
Example usage of the time series aggregation module (tsam)
Date: 29.04.2019

Author: Maximilian Hoffmann

Import pandas and the relevant time series aggregation class

In [ ]:
%load_ext autoreload
%autoreload 2
import copy

import matplotlib.pyplot as plt
import pandas as pd

import tsam
from tsam import ClusterConfig
from tsam.timeseriesaggregation import unstackToPeriods

%matplotlib inline

### Input data 

Read in time series from testdata.csv with pandas

In [ ]:
raw = pd.read_csv("testdata.csv", index_col=0)

Create a plot function for the temperature for a visual comparison of the time series

In [ ]:
def plotTS(data, periodlength, vmin, vmax, label="T [°C]"):
    fig, axes = plt.subplots(figsize=[6, 2], dpi=100, nrows=1, ncols=1)
    stacked, timeindex = unstackToPeriods(copy.deepcopy(data), periodlength)
    cax = axes.imshow(stacked.values.T, interpolation="nearest", vmin=vmin, vmax=vmax)
    axes.set_aspect("auto")
    axes.set_ylabel("Hour")
    plt.xlabel("Day")

    fig.subplots_adjust(right=1.2)
    cbar = plt.colorbar(cax)
    cbar.set_label(label)

### Hierarchical aggregation with medoid representation at hourly resolution

Initialize an aggregation class object with hierarchical as method for eight typical days

In [ ]:
resultMedoid = tsam.aggregate(
    raw,
    n_periods=8,
    period_hours=24,
    cluster=ClusterConfig(method="hierarchical", representation="medoid"),
)

Create the typical periods

In [ ]:
typPeriodsMedoid = resultMedoid.typical_periods

Predict original data

In [ ]:
predictedPeriodsMedoid = resultMedoid.reconstruct()

Get accuracy indicators

In [ ]:
resultMedoid.accuracy

### Hierarchical aggregation with maxoid representation at hourly resolution

In [ ]:
resultMaxoid = tsam.aggregate(
    raw,
    n_periods=8,
    period_hours=24,
    cluster=ClusterConfig(method="hierarchical", representation="maxoid"),
    rescale=False,
)

Create the typical periods

In [ ]:
typPeriodsMaxoid = resultMaxoid.typical_periods

Predict original data

In [ ]:
predictedPeriodsMaxoid = resultMaxoid.reconstruct()

Get accuracy indicators

In [ ]:
resultMaxoid.accuracy

### Hierarchical aggregation with mean representation and 10 typical days at hourly resolution

In [ ]:
resultMean = tsam.aggregate(
    raw,
    n_periods=20,
    period_hours=24,
    cluster=ClusterConfig(method="hierarchical", representation="mean"),
)

Create the typical periods

In [ ]:
typPeriodsMean = resultMean.typical_periods

Predict original data

In [ ]:
predictedPeriodsMean = resultMean.reconstruct()

Get accuracy indicators

In [ ]:
resultMean.accuracy

### Hierarchical aggregation with minmax representation and 10 typical days at hourly resolution

In [ ]:
resultMinmax = tsam.aggregate(
    raw,
    n_periods=20,
    period_hours=24,
    cluster=ClusterConfig(method="hierarchical", representation="minmax_mean"),
    rescale=False,
)

Create the typical periods

In [ ]:
typPeriodsMinmax = resultMinmax.typical_periods

Predict original data

In [ ]:
predictedPeriodsMinmax = resultMinmax.reconstruct()

Get accuracy indicators

In [ ]:
resultMinmax.accuracy

### Hierarchical aggregation with duration representation and 10 typical days at hourly resolution

In [ ]:
resultDuration = tsam.aggregate(
    raw,
    n_periods=20,
    period_hours=24,
    cluster=ClusterConfig(method="hierarchical", representation="duration"),
    rescale=False,
)

Create the typical periods

In [ ]:
typPeriodsDuration = resultDuration.typical_periods

Predict original data

In [ ]:
predictedPeriodsDuration = resultDuration.reconstruct()

Get accuracy indicators

In [ ]:
resultDuration.accuracy

### Comparison of the aggregations 
It was shown for the temperature, but both times all four time series have been aggregated. Therefore, we compare here also the duration curves  of the electrical load for the original time series, the aggregation with k-mean, and the hierarchical aggregation including peak periods.

In [ ]:
fig, axes = plt.subplots(figsize=[6, 2], dpi=100, nrows=1, ncols=1)
raw["Load"].sort_values(ascending=False).reset_index(drop=True).plot(label="Original")
predictedPeriodsMedoid["Load"].sort_values(ascending=False).reset_index(drop=True).plot(
    label="10 Medoids"
)
predictedPeriodsMaxoid["Load"].sort_values(ascending=False).reset_index(drop=True).plot(
    label="10 Maxoids"
)
predictedPeriodsMean["Load"].sort_values(ascending=False).reset_index(drop=True).plot(
    label="10 Centroids"
)
predictedPeriodsMinmax["Load"].sort_values(ascending=False).reset_index(drop=True).plot(
    label="10 Maxima"
)
predictedPeriodsDuration["Load"].sort_values(ascending=False).reset_index(
    drop=True
).plot(label="10 Duration")
plt.legend()
plt.xlabel("Hours [h]")
plt.ylabel("Duration Load [MW]")

In [ ]:
param = "GHI"

In [ ]:
plotTS(raw[param], 24, vmin=raw[param].min(), vmax=raw[param].max(), label=param)

In [ ]:
plotTS(
    predictedPeriodsMedoid[param],
    24,
    vmin=raw[param].min(),
    vmax=raw[param].max(),
    label=param,
)

In [ ]:
plotTS(
    predictedPeriodsMaxoid[param],
    24,
    vmin=raw[param].min(),
    vmax=raw[param].max(),
    label=param,
)

In [ ]:
plotTS(
    predictedPeriodsMean[param],
    24,
    vmin=raw[param].min(),
    vmax=raw[param].max(),
    label=param,
)

In [ ]:
plotTS(
    predictedPeriodsMinmax[param],
    24,
    vmin=raw[param].min(),
    vmax=raw[param].max(),
    label=param,
)

In [ ]:
plotTS(
    predictedPeriodsDuration[param],
    24,
    vmin=raw[param].min(),
    vmax=raw[param].max(),
    label=param,
)

In [ ]:
fig, axes = plt.subplots(figsize=[6, 2], dpi=100, nrows=1, ncols=1)
raw["Load"]["20100210":"20100218"].plot(label="Original")
predictedPeriodsMedoid["Load"]["20100210":"20100218"].plot(label="10 Medoids")
predictedPeriodsMaxoid["Load"]["20100210":"20100218"].plot(label="10 Maxoids")
predictedPeriodsMean["Load"]["20100210":"20100218"].plot(label="10 Centroids")
predictedPeriodsMinmax["Load"]["20100210":"20100218"].plot(label="10 Maxima")
predictedPeriodsDuration["Load"]["20100210":"20100218"].plot(label="10 Duration")
plt.legend()
plt.ylabel("Load [MW]")

### Validation

Check that the means of the original time series and the predicted ones are the same.

In [ ]:
raw.mean()

In [ ]:
predictedPeriodsMedoid.mean()

In [ ]:
predictedPeriodsMaxoid.mean()

In [ ]:
predictedPeriodsMean.mean()

In [ ]:
predictedPeriodsMinmax.mean()

In [ ]:
predictedPeriodsDuration.mean()